In [1]:
import os

In [2]:
%pwd

'/home/muhammed-shafeeh/AI_ML/waste_detection_yolo_v5/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/muhammed-shafeeh/AI_ML/waste_detection_yolo_v5'

In [72]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    data_validation_status_file: Path
    data_validation_required_files_dir: Path
    data_validation_all_required_files: list
    

In [73]:
from src.yoloproject.constants import *
from src.yoloproject.utils.common import read_yaml, create_directories



In [84]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.data_validation.root_dir])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([Path(config.root_dir)])

        data_validation_config = DataValidationConfig(
            root_dir=Path(config.root_dir),
            data_validation_status_file=Path(config.data_validation_status_file),
            data_validation_required_files_dir=Path(config.data_validation_required_files_dir),  
            data_validation_all_required_files=list(config.data_validation_all_required_files) # Expected files
        )

        return data_validation_config

In [85]:
from src.yoloproject.exception import AppException
import sys

In [86]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_data(self) -> bool:
        try:
            feature_store_path = self.config.data_validation_all_required_files  # Expected files
            actual_files = set(os.listdir(self.config.data_validation_required_files_dir))  # Files present in the directory

            # Check if all required files are present
            missing_files = [file for file in feature_store_path if file not in actual_files]
            validation_status = len(missing_files) == 0

            # Ensure the directory for status file exists
            self.config.root_dir.mkdir(parents=True, exist_ok=True)

            # Write validation status to file
            with open(self.config.root_dir.joinpath(self.config.data_validation_status_file), "w") as file:
                file.write(f"Validation status: {validation_status}\n")
                if not validation_status:
                    file.write(f"Missing files: {', '.join(missing_files)}\n")

            return validation_status

        except Exception as e:
            raise AppException(e, sys)

In [88]:
config_manager = ConfigurationManager()
data_validation_config = config_manager.get_data_validation_config()

data_validator = DataValidation(config=data_validation_config)
validation_status = data_validator.validate_data()